# <center> What makes a song a hit? 
    
### <center> Data Science in Practice
   Authors:
    
  

In [4]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json
pd.set_option('display.max_columns', None)

# Modelling Regression

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix , classification_report, mean_squared_error, r2_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix,average_precision_score

## Feature Engineering for Regression

Besides keeping all the music quality features we used in out classifier model, we try to understand which kinds of time-series data could be introduced to our regressor.  

Normally, the Spotify users may try to listen the songs which has already been popular. Therefore, we introduce the new features recording the stream in the last 5 appearances in the list and the position in the last list.  

However, if the popular song stays too long on the top lists, the user may consider it “out-of-fashion” and streams less. In order to take this case into consideration, we introduced two other features: 

Week-count: count how many times the song has been in the top 200 list 
Week-prev: when was the last time the song appeared in the top 200 list.  
Week: current week of the estimation, working together with the week-prev, we could observe if the song has felt out of the list in the past period. 

### Step 1 Feature Selection & Dynamic Features

In [ ]:
dfall_fe = dfall.copy()
dfall_fe = dfall_fe.drop(columns = ['URL', 'uri','track_href','analysis_url','type','Track Name'])
dfall_fe_id_values = dfall_fe.id.unique().tolist()
cdff = ['counts','index_prev','week_prev','pos_prev','stream_prev']
vdff = np.empty((len(dfall_fe_id_values),len(cdff)))
vdff[:] = np.nan
dfFeature = pd.DataFrame(vdff, 
                         index =dfall_fe_id_values, columns = cdff)
dfall_fe['hist'] = [0]*len(dfall_fe.index)
dfall_fe['new'] = [0]*len(dfall_fe.index)

In [ ]:
for i in range(dfall_fe.shape[0]):
    if np.isnan(dfFeature.at[dfall_fe.id[i],'counts']):
        dfall_fe.at[i,'new'] = 1
        dfall_fe.at[i,'hist'] = 1
        dfFeature.at[dfall_fe.id[i],'counts'] = 1
        dfFeature.at[dfall_fe.id[i],'stream_prev'] = dfall_fe.at[i,'Streams']
        dfFeature.at[dfall_fe.id[i],'stream_log1'] = dfall_fe.at[i,'Streams']
        dfFeature.at[dfall_fe.id[i],'stream_log2'] = dfall_fe.at[i,'Streams']
        dfFeature.at[dfall_fe.id[i],'stream_log3'] = dfall_fe.at[i,'Streams']
        dfFeature.at[dfall_fe.id[i],'stream_log4'] = dfall_fe.at[i,'Streams']
        dfall_fe.at[i,'week_prev'] = 0
        dfall_fe.at[i,'stream_prev'] = 0
        dfall_fe.at[i,'pos_prev'] = 200
        
    else:
        dfFeature.at[dfall_fe.id[i],'counts'] += 1
        dfall_fe.at[i,'week_prev'] = dfFeature.at[dfall_fe.id[i],'week_prev']
        dfall_fe.at[i,'stream_prev'] = dfFeature.at[dfall_fe.id[i],'stream_prev']
        dfall_fe.at[i,'stream_prev'] = dfFeature.at[dfall_fe.id[i],'stream_prev']
        dfall_fe.at[i,'pos_prev'] = dfFeature.at[dfall_fe.id[i],'pos_prev']
        dfall_fe.at[i,'hist'] = dfFeature.at[dfall_fe.id[i],'counts']
    dfFeature.at[dfall_fe.id[i],'index_prev'] = i
    dfFeature.at[dfall_fe.id[i],'week_prev'] = dfall_fe.at[i,'week']
    dfFeature.at[dfall_fe.id[i],'stream_log4'] = dfFeature.at[dfall_fe.id[i],'stream_log3']
    dfFeature.at[dfall_fe.id[i],'stream_log3'] = dfFeature.at[dfall_fe.id[i],'stream_log2']
    dfFeature.at[dfall_fe.id[i],'stream_log2'] = dfFeature.at[dfall_fe.id[i],'stream_log1']
    dfFeature.at[dfall_fe.id[i],'stream_log1'] = dfFeature.at[dfall_fe.id[i],'stream_prev']
    dfFeature.at[dfall_fe.id[i],'stream_prev'] = dfall_fe.at[i,'Streams']
    dfFeature.at[dfall_fe.id[i],'pos_prev'] = dfall_fe.at[i,'Position']
    dfall_fe.at[i,'stream_avg'] = 1/5*(dfall_fe.at[i,'Streams'] + dfFeature.at[dfall_fe.id[i],'stream_prev']
                                      + dfFeature.at[dfall_fe.id[i],'stream_log1']+ dfFeature.at[dfall_fe.id[i],'stream_log2']
                                      + dfFeature.at[dfall_fe.id[i],'stream_log3']+ dfFeature.at[dfall_fe.id[i],'stream_log4'])
    

### Step 2 Feature Augmentation

In [ ]:
features=['danceability','energy','key','loudness','mode' ,'speechiness','acousticness','instrumentalness','liveness','valence','tempo','new','week','week_prev','pos_prev','stream_prev','duration_ms']    
X_orig = dfall_fe[features]
from sklearn.preprocessing import StandardScaler
x = X_orig.values #returns a numpy array
std_scaler = StandardScaler()
x_scaled = std_scaler.fit_transform(x)

X_scaled = pd.DataFrame(x_scaled, columns = features) 

In [ ]:
def generate_polynormial_features_set(X_scaled, k ):
    x_scaled = X_scaled.to_numpy()
    features = X_scaled.columns.tolist()
    for order in range(2,k+1):
        features_tmp = features
        for i in range(len(features)):
            features_tmp[i]= features[i]+'_'+str(order)
            x_tmp = x_scaled**order
        X_tmp = pd.DataFrame(x_tmp, columns = features_tmp) 
        if order == 2:
            X_poly = X_tmp
        else:
            X_poly = pd.concat([X_poly, X_tmp], axis=1).reindex(X_poly.index)
    return X_poly

## Position Prediction – Use Regression to Classify the Top Songs 

One of the limitations in our classification model is that the unbalanced dataset will generate the estimation bias. Although we could use some sampling approaches to generate a balanced dataset, new bias could be introduced during the process.  

As the classification object “Top” is based on the value “Position”, one of the possible solutions is to change the classification problem to a regression problem on “Position”. The classification will be done based on the predicted “Position” value.  

Different from the approach in our classification model, as the position is a time-dependent value, the model will be week-based. Some dynamic features will be introduced to improve the accuracy of the time series approach.  

#### Linear Regression

In [ ]:
X = X_scaled
y = dfall['Position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Test Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('Train Mean squared error: %.2f'
      % mean_squared_error(y_train, y_pred_train))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

#### Polynomial Regression

In [ ]:
X_poly = generate_polynormial_features_set(X_scaled, 4)
X = pd.concat([X_scaled, X_poly], axis=1).reindex(X_scaled.index)
y = dfall['Position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Test Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('Train Mean squared error: %.2f'
      % mean_squared_error(y_train, y_pred_train))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

#### Full features regression

In [ ]:
features_sqrt = features
for i in range(len(features)):
    features_sqrt[i]= features[i]+'_sqrt'

x_sqrt = np.sqrt(np.abs(x_scaled))
X_sqrt = pd.DataFrame(x_sqrt, columns = features_sqrt) 

features_sin = features
for i in range(len(features)):
    features_sin[i]= features[i]+'_sin'

x_sin = np.sin(x_scaled)
X_sin = pd.DataFrame(x_sin, columns = features_sin) 

features_cos = features
for i in range(len(features)):
    features_cos[i]= features[i]+'_cos'

x_cos = np.cos(x_scaled)
X_cos = pd.DataFrame(x_cos, columns = features_cos) 

In [ ]:
X = pd.concat([X_scaled, X_poly, X_sqrt, X_sin, X_cos], axis=1).reindex(X_scaled.index)
y = dfall['Position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Test Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('Train Mean squared error: %.2f'
      % mean_squared_error(y_train, y_pred_train))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

#### Regression to Full features

In [ ]:
y_test_clf = pd.DataFrame(y_test)
y_test_clf['top'] = np.where(y_test_clf['Position']<dfall.Position.quantile(0.2), 1, 0)
y_pred_clf = pd.DataFrame(data = y_pred, columns = ['Position'])
y_pred_clf['top'] = np.where(y_pred_clf['Position']<dfall.Position.quantile(0.2), 1, 0)

mat = confusion_matrix(y_test_clf['top'], y_pred_clf['top'])
s1=sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
          xticklabels=['No','Yes'],
          yticklabels=['No','Yes'] )
plt.xlabel('true label')
plt.ylabel('predicted label')
bottom, top = s1.get_ylim()
s1.set_ylim(bottom + 0.5, top - 0.5)

print("Accuracy Score:",accuracy_score(y_test_clf['top'],y_pred_clf['top']))
print("Recall Score:",recall_score(y_test_clf['top'],y_pred_clf['top'],labels=[1,0]))
print("Precision Score:",precision_score(y_test_clf['top'],y_pred_clf['top'],labels=[1,0]))

## Regression based on Streams

Based on the current business model of Spotify, streaming number is the decisive factor for the singer's profit. We could therefore predict the future number of streams as the estimative indicator for the singer's profit from spotify. 

The problem could be defined as a regression problem with continious output variable rather than binary-classfication. Therefore, We need to choose different models with different feature set for this problem. The coefficient of determination is chosen as the measure on the performance of our regressor. 

### Linear Regression

In [65]:
X = X_scaled
y = dfall['Streams']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Test Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('Train Mean squared error: %.2f'
      % mean_squared_error(y_train, y_pred_train))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))


Coefficients: 
 [ 4.48258270e+04 -7.08489351e+04 -8.33593504e+03  3.02753903e+04
 -6.97697564e+03 -2.78033578e+04 -1.11905228e+04  3.76281273e+03
  1.15042939e+04 -3.03932010e+03 -5.09047953e+02  1.08969128e+06
 -2.68470009e+04  5.53897483e+04 -9.74545814e+04  1.74375184e+06
 -2.08165968e+04]
Test Mean squared error: 1624025882432.18
Train Mean squared error: 1637912832055.08
Coefficient of determination: 0.63


### Polynormial Regression

In [66]:
X_poly = generate_polynormial_features_set(X_scaled, 4)
X = pd.concat([X_scaled, X_poly], axis=1).reindex(X_scaled.index)
y = dfall['Streams']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Test Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('Train Mean squared error: %.2f'
      % mean_squared_error(y_train, y_pred_train))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

Coefficients: 
 [ 3.57387558e+04 -7.05321814e+04  7.22694469e+04  3.21968940e+04
  2.57659912e+01 -1.29834188e+04  3.09494942e+04 -5.73547758e+04
 -3.48121321e+04 -5.41960366e+04  1.20207032e+04  2.12861792e+03
 -4.74083147e+05  9.79908739e+05 -7.41430504e+04  1.48431858e+06
 -9.59323461e+03 -2.47087225e+03 -1.41871489e+04 -1.47698599e+05
  1.91165014e+04 -1.01823997e+01 -3.39729074e+04  3.29200641e+03
  2.41114014e+04  4.23610741e+04 -4.82453232e+04 -1.42344535e+04
  4.82583737e+03  1.04653798e+06 -1.24275821e+06 -2.54141628e+05
 -8.39486218e+04 -7.92186810e+03 -7.13541992e+03 -3.96726849e+03
 -4.70087253e+04 -1.91961035e+03  2.97897491e+01  1.60568280e+04
 -3.05612305e+04 -2.25875000e+03 -6.03628101e+03  1.82800033e+04
 -1.01013775e+04  1.30693823e+04  1.78086457e+05 -7.06102698e+05
 -9.13294762e+04  8.45706030e+03  4.28005371e+02 -2.31507373e+03
 -2.27126721e+03  4.74250677e+04 -6.68660156e+02 -2.19549484e+01
 -1.90813281e+03  6.54166406e+03  6.92500000e+01 -1.53169922e+02
  9.46396

### Regression on Full features

In [67]:
X = pd.concat([X_scaled, X_poly, X_sqrt, X_sin, X_cos], axis=1).reindex(X_scaled.index)
y = dfall['Streams']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Test Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('Train Mean squared error: %.2f'
      % mean_squared_error(y_train, y_pred_train))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

Coefficients: 
 [ 1.97340796e+06  8.16513070e+05  2.40734265e+05  3.93371666e+05
  1.83704865e+02  2.07560739e+05  6.04804222e+06 -7.23048481e+03
 -4.47213613e+05 -8.67869697e+05 -1.12736586e+06  3.74953450e+03
 -2.72652038e+07  3.88448508e+07  2.83289802e+07  2.63375767e+06
  5.23351316e+04  5.44907988e+05  3.11671277e+05 -8.17482919e+06
 -8.94300308e+04 -7.25979614e+01 -5.54107012e+04 -3.15587904e+06
  4.22960347e+04 -3.03904867e+05  3.10165238e+06  3.32689701e+06
  8.50065497e+03 -2.96519474e+06 -2.04282064e+07 -5.60291149e+07
 -1.10789263e+06 -2.11529608e+03 -2.79011301e+05 -1.20539985e+05
 -7.56498376e+04 -7.17148962e+04  2.12394623e+02 -1.12223491e+04
 -9.09825059e+05 -4.18072852e+03  1.47156338e+05  1.38324176e+05
  1.57222474e+05  2.30215535e+04  3.98875533e+06 -6.26264179e+06
 -4.27305665e+06  1.83240140e+05 -6.25904150e+03 -6.35507747e+04
 -3.10661603e+04  6.37372594e+05 -7.20333203e+03 -1.56533905e+02
  2.03636523e+03  2.91100552e+05  1.06671875e+02 -1.40884971e+04
 -2.13315

### Support Vector Regressor

In [72]:
X_poly = generate_polynormial_features_set(X_scaled, 4)
X = pd.concat([X_scaled, X_poly], axis=1).reindex(X_scaled.index)
y = dfall['Streams']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = svm.SVR()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

# The mean squared error
print('Test Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('Train Mean squared error: %.2f'
      % mean_squared_error(y_train, y_pred_train))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

C:\Users\ydi\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Test Mean squared error: 5405677940613.91
Train Mean squared error: 4699251964268.54
Coefficient of determination: -0.12
